In [10]:
#!pip3 install uvicorn fastapi pyngrok


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# 서버 관리용 fastapi 의존 라이브러리
import uvicorn

# fastapi 라이브러리
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
from fastapi import File, UploadFile
from fastapi.responses import JSONResponse, HTMLResponse


# 인터페이스 데이터 관리를 위한 라이브러리
from pydantic import BaseModel

# 모델 Load 용 라이브러리
import PCB_Product_Model
import PCB_Defect_Model

# 데이터 분석용 라이브러리
import pandas as pd
import numpy as np
import os
import cv2
from ultralytics import YOLO

# ngrok을 활용하여 분석서버를 Global하게 띄우기 위한 라이브러리
import nest_asyncio
from pyngrok import ngrok
import uvicorn

In [3]:
# CORS issue 때문에 미들웨어 추가

from fastapi.middleware.cors import CORSMiddleware
origins = ["*"]

app = FastAPI(title="ML API")

# CORS 미들웨어 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 origin 허용
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],
)

### 1. 모델 불러오기

In [ ]:
# 제품 분류 모델 (CNN)

In [10]:
model_path = 'pcb_prod_model_0708.h5'
class_path = 'pcb_class_dict_0708.json'

pcb_prod_model = PCB_Product_Model.Model_load(model_path, class_path)


In [ ]:
# 제품 양불 판정 모델 (YOLO)

In [ ]:


pcb_defect_model = PCB_Defect_Model

### 2. 예측 시뮬레이션

In [5]:
@app.get("/", response_class=HTMLResponse)
async def main():
    content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Image Upload</title>
        <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    </head>
    <body>
        <h1>Upload an Image</h1>
        <form id="uploadForm">
            <input type="file" id="fileInput" name="file"><br><br>
            <button type="button" id="uploadButton">Upload</button>
        </form>
        <div id="result"></div>

        <script>
            $(document).ready(function() {
                $('#uploadButton').click(function() {
                    var formData = new FormData();
                    var fileInput = $('#fileInput')[0];
                    if (fileInput.files.length === 0) {
                        alert('Please select a file');
                        return;
                    }

                    formData.append('file', fileInput.files[0]);

                    $.ajax({
                        url: '/predict',
                        type: 'POST',
                        data: formData,
                        contentType: false,
                        processData: false,
                        success: function(response) {
                            console.log(response)
                            //$('#result').html('<p>' + response.message + '</p><p>Filename: ' + response.filename + '</p>');
                        },
                        error: function(response) {
                            //$('#result').html('<p>' + response.responseJSON.error + '</p>');
                        }
                    });
                });
            });
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=content)


@app.post("/predict")
async def upload_image(file: UploadFile = File(...)):

    contents = await file.read()

    # 이미지 읽기
    nparr = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    prod_type = pcb_prod_model.predict_image(False, img)

    prod_defect_dict = pcb_defect_model.predict_defect(img)

    error_type_list = prod_defect_dict['defect_name_list']
    defect_box_list = prod_defect_dict['image']
    
    # img_str = ndarray_to_base64(defect_image)
    
    result = {
        "product_type": prod_type,
        "error_type_list": error_type_list,
        "image": defect_box_list
    }

    return result


### 3.

In [6]:
# colab 서버 주소는 실행 될 때마다 바뀌기 때문에 ngrok이라는 것을 활용하여서 고정적인 url로 들어올 수 있는 서버를 만들 수 있다. 아래의 코드를 활용하면 된다.

import nest_asyncio
from pyngrok import ngrok
import uvicorn

AUTH_TOKEN = "2hRkhpGbfhVbsBcpYewFZne3I4z_24P9h5EchPjzVTMJe6zEL"
ngrok.set_auth_token(AUTH_TOKEN)
ngrokTunnel = ngrok.connect(8000)
print("공용 URL", ngrokTunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)



공용 URL https://b8a1-118-34-210-82.ngrok-free.app


INFO:     Started server process [13896]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     27.0.238.70:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     211.231.103.93:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     27.0.238.70:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     211.231.103.91:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     118.34.210.29:0 - "POST / HTTP/1.1" 405 Method Not Allowed
1/1 [==============================] - 0s 135ms/step
INFO:     118.34.210.25:0 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 29ms/step
INFO:     118.34.210.25:0 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 29ms/step
INFO:     118.34.210.29:0 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 31ms/step
INFO:     118.34.210.25:0 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 29ms/step
INFO:     118.34.210.25:0 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 31ms/step
INFO:     118.34.210

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [13896]
